In [46]:
import pandas as pd 
import numpy as np
df = pd.read_csv("data/output/result.csv", header=None)
df.columns = ['accuracy', 'explanation', 'positive counterexamples', 'query', 'random words checked', 'terminate', 'time', 'time learner', 'time verifier', 'total counterexamples']
df.head()

,accuracy,explanation,positive counterexamples,query,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,1.0,(not (or eggs breathes)),0.400000,breathes = 0,234.0,1.0,29.158047,1.553447,27.601642,5.0
1,1.0,(not (or eggs breathes)),0.500000,breathes = 0,240.0,1.0,30.232830,1.738493,28.490781,6.0
2,1.0,(not (or eggs breathes)),0.500000,breathes = 0,249.0,1.0,31.423432,1.763439,29.656554,6.0
3,1.0,(not (or eggs breathes)),0.500000,breathes = 0,266.0,1.0,33.850034,1.801139,32.045468,6.0
4,1.0,(not (or eggs breathes)),0.428571,breathes = 0,268.0,1.0,34.315431,2.108706,32.202708,7.0


In [47]:
# take median
group_list = ['query']
df_med = df.groupby(group_list).mean()
df_med.reset_index(inplace=True)
df_med.head()

,query,accuracy,positive counterexamples,random words checked,terminate,time,time learner,time verifier,total counterexamples
0,aquatic = 0,0.693182,0.447527,363.375,0.0,121.935098,88.981492,31.946161,14.250
1,backbone = 1,0.750000,0.458976,257.125,0.0,121.031171,94.880668,25.142729,15.250
2,breathes = 0,1.000000,0.478571,265.375,1.0,33.479399,1.876691,31.599223,6.250
3,eggs = 0,0.863636,0.477289,362.625,0.0,121.186851,87.557769,32.621760,14.125
4,legs <= 0.2,0.488636,0.451809,357.625,0.0,121.030885,84.551095,35.472670,13.000


In [48]:
# find representative explanation
query_explanations = {}
for key, item in df.groupby(['query'], as_index=False):
    item.reset_index(inplace=True, drop=True)
    explanation = item.iloc[item.index[(item['accuracy']-df_med['accuracy'][(df_med['query'] == key)].item()).abs().argsort()][0]]['explanation']
    query_explanations[key] = explanation

In [49]:
df_output = df_med
df_output = df_output.drop("terminate", axis=1)
df_output['explanation'] = np.nan 
for key in query_explanations:
    df_output['explanation'][df_output['query'] == key] = query_explanations[key]

# reorganise columns
df_output = df_output[['query', 'explanation', 'accuracy', 'time', 'time learner', 'time verifier', 'random words checked', 'total counterexamples', 'positive counterexamples']]
df_output.head()

,query,explanation,accuracy,time,time learner,time verifier,random words checked,total counterexamples,positive counterexamples
0,aquatic = 0,(not (or aquatic (and tail (not backbone)))),0.693182,121.935098,88.981492,31.946161,363.375,14.250,0.447527
1,backbone = 1,(and backbone (and (or hair domestic) (or mil...,0.750000,121.031171,94.880668,25.142729,257.125,15.250,0.458976
2,breathes = 0,(not (or eggs breathes)),1.000000,33.479399,1.876691,31.599223,265.375,6.250,0.478571
3,eggs = 0,(not (or eggs (and breathes (not (and hair aq...,0.863636,121.186851,87.557769,32.621760,362.625,14.125,0.477289
4,legs <= 0.2,(and backbone (and (or feathers breathes) (no...,0.488636,121.030885,84.551095,35.472670,357.625,13.000,0.451809


In [41]:
df_output.to_csv("/home/bishwamittra/Dropbox/trustable_explanations/result.csv", index = False)